In [ ]:
import pandas as pd
Test_trans = pd.read_csv('./test_operation_round2.csv')

Test_trans['device2'] = Test_trans['device2'].str.strip()
Test_trans['device1'] = Test_trans['device1'].str.strip()
Test_trans['device_'] = Test_trans['device1']+'_'+Test_trans['device2']
dev = Test_trans['device_'].value_counts()
dev = dev[dev>4].index
uuuid = []

for i,d in enumerate(dev):
    print(i/len(dev))
    MI_6 = Test_trans[Test_trans['device_']==d]
    nn = MI_6.sort_values('geo_code').drop_duplicates(['geo_code','UID'])#.set_index(['device1','geo_code'])
    geo = nn['geo_code'].value_counts()
    geo = geo[geo>4].index
    aa = MI_6[MI_6['geo_code'].isin(geo)].groupby(['geo_code','day','mode','wifi'])['UID']
    hh = aa.nunique()
    en = hh[(hh>4)].reset_index()

    for i in range(en.shape[0]):
        bb = MI_6[(MI_6['wifi']== en.loc[i]['wifi'] )&(MI_6['geo_code']== en.loc[i]['geo_code'] )&(MI_6['day']==en.loc[i]['day'])&(MI_6['mode']==en.loc[i]['mode'])][['UID','day','time','device1','device2','geo_code']]
        uuuid = uuuid + bb['UID'].value_counts().index.tolist()

b = list(set(uuuid))
print(len(b))
#base38_1599_opera_wifi最高分：0.54564
#9的时候，479， 合并后1589； 8的时候555，合并还是1589；7的时候626，合并1599；6的时候，626，合并仍然1599；5 的时候721，合并1626；4的时候758，合并1636

In [ ]:
import pandas as pd
Test_trans = pd.read_csv('./test_transaction_round2.csv')
Test_trans['device2'] = Test_trans['device2'].str.strip()
Test_trans['device1'] = Test_trans['device1'].str.strip()
Test_trans['device_'] = Test_trans['device1']+'_'+Test_trans['device2']
dev = Test_trans['device_'].value_counts()

dev = dev[dev>4].index
uuuid = []

for d in dev:
    MI_6 = Test_trans[Test_trans['device_']==d]
    nn = MI_6.sort_values('geo_code').drop_duplicates(['geo_code','UID'])#.set_index(['device1','geo_code'])
    geo = nn['geo_code'].value_counts()
    geo = geo[geo>4].index
    hh = MI_6[MI_6['geo_code'].isin(geo)].groupby(['geo_code','day','merchant'])['UID'].nunique()
    en = hh[hh>4].reset_index()

    for i in range(en.shape[0]):
        bb = MI_6[(MI_6['geo_code']== en.loc[i]['geo_code'] )&(MI_6['day']==en.loc[i]['day'])&(MI_6['merchant']==en.loc[i]['merchant'])][['UID','channel','day','time','device1','device2','geo_code','merchant']]
        uuuid = uuuid + bb['UID'].value_counts().index.tolist()

bb_ = list(set(uuuid))
print(len(bb_))

In [ ]:
bbb = list(set(bb_).union(set(b)))
print(len(bbb))
fi_uid = pd.DataFrame(bbb,columns = ['UID'])
fi_uid.head()#1362

In [ ]:
import pandas as pd
Test_trans = pd.read_csv('./test_transaction_round2.csv')
Test_tag = pd.read_csv('./lgb_baseline_12-12-22-05.csv')  # 测试样本

test_rule_uid = pd.DataFrame(fi_uid.UID.unique())
pred_data_rule = Test_tag.merge(test_rule_uid,left_on ='UID',right_on =0, how ='left')
pred_data_rule['Tag'][(pred_data_rule[0]>0)] =1 # 这个系数还需要调整
pred_data_rule[['UID', 'Tag']].to_csv('result.csv', index=False)
a = pred_data_rule[pred_data_rule['Tag'] == 1]['UID'].values
len(a)